<a href="https://colab.research.google.com/github/sriksmachi/ishara/blob/main/Neural_Nets_Project_Starter_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [4]:
# !pip install scipy
# !pip install Pillow
# !pip install imageio
# !pip install tensorflow
# !pip install matplotlib
#!pip install opencv-python
# !pip install gdown

In [5]:
import numpy as np
import os
import imageio
import datetime
import os
import cv2
import pathlib
import gdown
from scipy import misc

We set the random seed so that the results don't vary drastically.

In [6]:
import random as rn
from keras import backend as K
import tensorflow as tf

rn.seed(30)
np.random.seed(30)
tf.compat.v1.random.set_random_seed(30)

In [7]:
print(tf.__version__)

2.4.1


In [8]:
!python --version

Python 3.7.10


In [9]:
!nvidia-smi

Sun Apr 25 17:05:38 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Load Data

In [10]:
!gdown 'https://drive.google.com/uc?export=download&id=1kl5iauN3iVCPnSHys0xvTUnAd41CNu3S'

Downloading...
From: https://drive.google.com/uc?export=download&id=1kl5iauN3iVCPnSHys0xvTUnAd41CNu3S
To: /content/Project_data.zip
1.71GB [01:31, 18.6MB/s]


In [11]:
!unzip /content/Project_data.zip

Streaming output truncated to the last 5000 lines.
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00050.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00052.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00054.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00056.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00058.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00060.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00062.png  
   creating: Project_data/train/WIN_20180926_17_35_12_Pro_Thumbs_Down_new/
 extracting: Project_data/train/WIN_20180926_17_35_12_Pro_Thumbs_Down_new/WIN_20180926_17_35_12_Pro_00001.png  
 extracting: Project_data/t

In [17]:
## Use this to load the data if the data exists on gdrive and using google colab for 

data_dir="Project_data/"
data_dir_train = "Project_data/train/"
data_dir_val = 'Project_data/val/'

train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())


In [18]:
# ## Use this to load the data if the data exists on Azure ML
# # azureml-core of version 1.0.72 or higher is required
# from azureml.core import Workspace, Dataset

# subscription_id = '925a5ad8-a21a-48c0-92a2-5a8a2a4dfc46'
# resource_group = 'machinelearning-workbench'
# workspace_name = 'sriks-azureml'

# workspace = Workspace(subscription_id, resource_group, workspace_name)

# dataset = Dataset.get_by_name(workspace, name='gesture_ds')

# ## Uncommenting the below will initiate the download, set Overwrite=True if overwriting existing files is intended. 
# # doc: https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.filedataset?view=azure-ml-py#download-target-path-none--overwrite-false-
# dataset.download(target_path='Project_data/', overwrite=False)

# train_path = 'Project_data/train'
# val_path = 'Project_data/val'

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [19]:

train_doc = np.random.permutation(open(data_dir+'train.csv').readlines())
val_doc = np.random.permutation(open(data_dir+'val.csv').readlines())

## Visualization

Let us plot some sample images and run some transformations on the image so see the impact.

In [20]:
# # ## Pick some random sequence
# import matplotlib.pyplot as plt


# random_sequence = train_doc[np.random.randint(len(train_doc))].strip().split(';')[0]+'/'
# images = os.listdir(data_dir_train + random_sequence)
# # Create a code to visualize one instance of all the 30 images present in the sequence
# plt.figure(figsize=(10, 10))
# i = 0
# random_images = []
# for img in images:
#   ax = plt.subplot(6, 5, i + 1)
#   i = i + 1
#   random_images.append(os.path.join(data_dir_train, random_sequence, img))
#   img_bgr = cv2.imread(os.path.join(data_dir_train, random_sequence, img))
#   img_mp = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB) # This is needed because opencv uses BGR convention and matplotlib uses RGB
#   imgplot= plt.imshow(img_mp)
#   plt.axis("off")
#   plt.plot()

In [21]:
# # Random Image - Original
# random_image = random_images[np.random.randint(30)]
# image_bgr = cv2.imread(random_image)
# plt.imshow(cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB))
# print(image_bgr.shape)

In [22]:
# ## Cropped Image Sample
# crop_img = image_bgr[10:120, 10:160]
# plt.imshow(cv2.cvtColor(crop_img, cv2.COLOR_BGR2RGB))
# print(crop_img.shape)

In [23]:
# # Resize Image Sample
# dim = (120, 120)
# resized_img = cv2.resize(crop_img, dim, interpolation = cv2.INTER_AREA)
# plt.imshow(cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB))
# print(resized_img.shape)

In [24]:
# # Normalized Image
# plt.imshow(cv2.cvtColor(cv2.normalize(resized_img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F), cv2.COLOR_BGR2RGB))

## Generator

This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [25]:
def crop_image(image, size = 10):
     # cropping the image
    image_x = image.shape[0]
    image_y = image.shape[1]
    return image[size:image_x, size:image_y]

def normalize_minmax(image_resized, batch_data, folder, idx):
    batch_data[folder,idx,:,:,0] = cv2.normalize(image_resized[:,:,0], None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    batch_data[folder,idx,:,:,1] = cv2.normalize(image_resized[:,:,1], None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    batch_data[folder,idx,:,:,2] = cv2.normalize(image_resized[:,:,2], None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    return batch_data   

def crop_and_normalize(image,batch_data, folder, idx, y=120,z=120):
    
    #crop the images and resize them. Note that the images are of 2 different shape 
    #and the conv3D will throw error if the inputs in a batch have different shapes
    image_cropped = crop_image(image)
    image_resized = cv2.resize(image_cropped, (y,z), interpolation = cv2.INTER_AREA)
                    
    # using min max normalization.
    pending_batch_data = normalize_minmax(image_resized, batch_data, folder, idx)
    return pending_batch_data


def create_batch_data(t, source_path, img_idx, folder, batch, batch_size, batch_data, batch_labels):
    imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
    for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    
        image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+
                           imgs[item]).astype(np.float32)
                    
        # using min max normalization.
        batch_data = crop_and_normalize(image, batch_data, folder, idx)
                    
    batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
    return batch_data, batch_labels

def generator(source_path, folder_list, dim=(120,120), batch_size=30, ablation=None,samples=[5,10,15]):
    print( 'Source path = ', source_path, '; batch size =', batch_size, 'dim=', dim)
    x = len(samples) # number of taken from each video
    img_idx = samples #create a list of image numbers you want to use for a particular video
    y = dim[0] # image dim
    z = dim[1] # image dim
    while True:
        if ablation is not None: 
            t = np.random.permutation(folder_list[:ablation])
        else:
            t = np.random.permutation(folder_list)
        num_batches =  len(t) // batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size, 5)) # batch_labels is the one hot representation of the output
            
            for folder in range(batch_size): # iterate over the batch_size
                
                ## get images from the folder
                batch_data_labels = create_batch_data(t, source_path, img_idx, folder, batch, batch_size, batch_data, batch_labels)
                batch_data = batch_data_labels[0]
                batch_labels = batch_data_labels[1]
                
            yield batch_data, batch_labels 
            
        # The length of the folder list could leave some residue folders, the below code deals with it
        # Number of pending batches
        pending_batches = len(t) % batch_size
        
        pending_batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
        pending_batch_labels = np.zeros((batch_size, 5)) # batch_labels is the one hot representation of the output
        
        for folder in range(pending_batches): # iterate over the batch_size

            # getimages from the folder
            pending_batch_data_labels = create_batch_data(t, source_path, img_idx, folder, num_batches, batch_size, pending_batch_data, pending_batch_labels)
            pending_batch_data = pending_batch_data_labels[0]
            pending_batch_labels = pending_batch_data_labels[1]       
            
            
        yield pending_batch_data, pending_batch_labels
            

# g = generator(data_dir_train, train_doc)
# next(g)

In [26]:
# g = generator(data_dir_train, train_doc, (120,120), 30, ablation=None, samples=30)
# p = 0
# for k in g:
#     p+=1
# print(p)
# # next(g)

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

## Model

#### Objective

Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

### 3D Convolution

We will build the model using 3D Convolution Architecture

In [27]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here
samples_per_image=[0,3,5,10,13,15,20,22,25,29]

# Building a 3 D Convolution model.
model = Sequential()
model.add(Conv3D(16, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(len(samples_per_image),120,120,3)))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [28]:
optimiser =  'sgd' #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 8, 118, 118, 16)   1312      
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 4, 59, 59, 16)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 2, 57, 57, 32)     13856     
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 1, 28, 28, 32)     0         
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               6422784   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               6

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

### Ablation Experiment

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`. 
We will fit the model with 1 epoch just to validate if the model is working.
Notice I have not added any callbacks yet, we will do it in the later sections

In [29]:
ablation_size=10
num_epochs = 1
batch_size=30


num_train_sequences = len(train_doc[:ablation_size])
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc[:ablation_size])
print('# validation sequences =', num_val_sequences)

print ('# epochs =', num_epochs)


if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

print('steps_per_epoch: {}, validation_steps: {}'.format(steps_per_epoch, validation_steps))

train_generator = generator(data_dir_train, train_doc, dim=(120,120), batch_size=batch_size, ablation=ablation_size, samples=samples_per_image)
val_generator = generator(data_dir_val, val_doc, dim=(120,120), batch_size=batch_size, ablation=ablation_size, samples=samples_per_image)

# training sequences = 10
# validation sequences = 10
# epochs = 1
steps_per_epoch: 1, validation_steps: 1


In [30]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=[], validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=2, initial_epoch=0)

Source path =  Project_data/train/ ; batch size = 30 dim= (120, 120)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


1/1 [==============================] - 34s 34s/step - loss: 0.5681 - categorical_accuracy: 0.7000 - val_loss: 0.7037 - val_categorical_accuracy: 0.1000


The model seems to be working well, there are no errors. 

### Overfitting on training data. 

Let us overfit on the training data to see if the model is able to learn from the data. We are going to use less data and run for more epochs and see if the model is able to improve the accuracy and reduce the loss.

In [31]:
ablation_size=200
batch_size=50
num_epochs=10

# Lets us see the input to the model
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc[:ablation_size])
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc[:ablation_size])
print('# validation sequences =', num_val_sequences)
print ('# epochs =', num_epochs)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

print('steps_per_epoch: {}, validation_steps: {}'.format(steps_per_epoch, validation_steps))

train_generator = generator(data_dir_train, train_doc, dim=(120,120), batch_size=batch_size, ablation=ablation_size, samples=samples_per_image)
val_generator = generator(data_dir_val, val_doc, dim=(120,120), batch_size=batch_size, ablation=ablation_size, samples=samples_per_image)

# training sequences = 200
# validation sequences = 100
# epochs = 10
steps_per_epoch: 4, validation_steps: 2


In [32]:
history = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=[], validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train/ ; batch size = 50 dim= (120, 120)
Epoch 1/10
4/4 [==============================] - 6s 2s/step - loss: 1.8656 - categorical_accuracy: 0.2650 - val_loss: 1.7045 - val_categorical_accuracy: 0.1800
Epoch 2/10
4/4 [==============================] - 6s 2s/step - loss: 1.2140 - categorical_accuracy: 0.2200 - val_loss: 1.7544 - val_categorical_accuracy: 0.1800
Epoch 3/10
4/4 [==============================] - 5s 2s/step - loss: 1.1894 - categorical_accuracy: 0.1450 - val_loss: 1.5752 - val_categorical_accuracy: 0.2700
Epoch 4/10
4/4 [==============================] - 5s 2s/step - loss: 1.1186 - categorical_accuracy: 0.2600 - val_loss: 1.5616 - val_categorical_accuracy: 0.3300
Epoch 5/10
4/4 [==============================] - 5s 2s/step - loss: 1.1199 - categorical_accuracy: 0.2650 - val_loss: 1.5613 - val_categorical_accuracy: 0.3400
Epoch 6/10
4/4 [==============================] - 6s 2s/step - loss: 1.4104 - categorical_accuracy: 0.4050 - val_loss: 1.4857 

The training loss reduced and accuracy increased, the model is able to learn well. 
The validation loss is high and accuracy is low which is expected at the moment as we did not use the complete model.

### Callbacks

Here we define few callbacks which we will use later when we fit the complete model.

In [33]:
curr_dt_time = datetime.datetime.now()
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.001)

ES = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

callbacks_list = [LR, ES]

### Final Experiment

In [34]:
ablation_size=None
batch_size=30
num_epochs=20

# Lets us see the input to the model
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
print ('# epochs =', num_epochs)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

print('steps_per_epoch: {}, validation_steps: {}'.format(steps_per_epoch, validation_steps))

train_generator = generator(data_dir_train, train_doc, dim=(120,120), batch_size=batch_size, ablation=ablation_size, samples=samples_per_image)
val_generator = generator(data_dir_val, val_doc, dim=(120,120), batch_size=batch_size, ablation=ablation_size, samples=samples_per_image)

# training sequences = 663
# validation sequences = 100
# epochs = 20
steps_per_epoch: 23, validation_steps: 4


In [35]:
history = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train/ ; batch size = 30 dim= (120, 120)
Epoch 1/20
23/23 [==============================] - 17s 781ms/step - loss: 1.3245 - categorical_accuracy: 0.3783 - val_loss: 1.1046 - val_categorical_accuracy: 0.3667
Epoch 2/20
23/23 [==============================] - 16s 738ms/step - loss: 1.2489 - categorical_accuracy: 0.4623 - val_loss: 1.1112 - val_categorical_accuracy: 0.3833
Epoch 3/20
23/23 [==============================] - 17s 768ms/step - loss: 1.1831 - categorical_accuracy: 0.4841 - val_loss: 1.0232 - val_categorical_accuracy: 0.4083
Epoch 4/20
23/23 [==============================] - 17s 744ms/step - loss: 0.9954 - categorical_accuracy: 0.5565 - val_loss: 0.9216 - val_categorical_accuracy: 0.4917
Epoch 5/20
23/23 [==============================] - 17s 755ms/step - loss: 0.9110 - categorical_accuracy: 0.6116 - val_loss: 0.9082 - val_categorical_accuracy: 0.5000
Epoch 6/20
23/23 [==============================] - 17s 769ms/step - loss: 0.8649 - categorical_

### CNN + RNN Model using Transfer Learning

In [1]:
from tensorflow.keras.applications.resnet50 import ResNet50
import tensorflow as tf
from tensorflow.keras.layers import Flatten,Dense,TimeDistributed,GRU,Conv2D,MaxPooling2D
from tensorflow.keras.models import Sequential

In [2]:
resnet = ResNet50(include_top=False,weights='imagenet',input_shape=(120,120,3))
for layer in resnet.layers[:-10]:
    layer.trainable=False

cnn = Sequential([resnet])
cnn.add(Conv2D(16, kernel_size=(3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(120,120,3)))
# cnn.add(MaxPooling2D(pool_size=(2, 2)))
cnn.add(Conv2D(32, kernel_size=(2, 2), activation='relu', kernel_initializer='he_uniform'))
# cnn.add(MaxPooling2D(pool_size=(2, 2)))

cnn.add(Flatten())
cnn.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
cnn.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))

model = Sequential()
model.add(TimeDistributed(cnn,input_shape=(10,120,120,3)))
model.add(GRU(16,input_shape=(None,30,256),return_sequences=True))
model.add(GRU(8))
model.add(Dense(5,activation='softmax'))


94773248/94765736 [==============================] - 1s 0us/step


In [3]:
model.summary()

NameError: ignored

In [53]:
optimiser =  'sgd' #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_3 (TimeDist (None, 10, 256)           23958960  
_________________________________________________________________
gru_6 (GRU)                  (None, 10, 16)            13152     
_________________________________________________________________
gru_7 (GRU)                  (None, 8)                 624       
_________________________________________________________________
dense_9 (Dense)              (None, 5)                 45        
Total params: 23,972,781
Trainable params: 4,850,733
Non-trainable params: 19,122,048
_________________________________________________________________
None


### Ablation Experiment

In [58]:
ablation_size=100
batch_size=30
num_epochs=1

num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
print ('# epochs =', num_epochs)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

print('steps_per_epoch: {}, validation_steps: {}'.format(steps_per_epoch, validation_steps))

train_generator = generator(data_dir_train, train_doc, dim=(120,120), batch_size=batch_size, ablation=ablation_size, samples=list(range(0,30,3)))
val_generator = generator(data_dir_val, val_doc, dim=(120,120), batch_size=batch_size, ablation=ablation_size, samples=list(range(0,30,3)))

# training sequences = 663
# validation sequences = 100
# epochs = 10
steps_per_epoch: 23, validation_steps: 4


In [59]:
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=[], validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train/ ; batch size = 30 dim= (120, 120)
Epoch 1/10
23/23 [==============================] - 30s 1s/step - loss: 1.4793 - accuracy: 0.2812 - val_loss: 1.3961 - val_accuracy: 0.3167
Epoch 2/10
23/23 [==============================] - 30s 1s/step - loss: 1.4449 - accuracy: 0.3507 - val_loss: 1.3735 - val_accuracy: 0.1250
Epoch 3/10
23/23 [==============================] - 30s 1s/step - loss: 1.4123 - accuracy: 0.3580 - val_loss: 1.4122 - val_accuracy: 0.1333
Epoch 4/10
23/23 [==============================] - 29s 1s/step - loss: 1.3676 - accuracy: 0.3826 - val_loss: 1.4628 - val_accuracy: 0.3333
Epoch 5/10
23/23 [==============================] - 30s 1s/step - loss: 1.3426 - accuracy: 0.4087 - val_loss: 1.3712 - val_accuracy: 0.1917
Epoch 6/10
23/23 [==============================] - 30s 1s/step - loss: 1.3498 - accuracy: 0.4319 - val_loss: 1.3530 - val_accuracy: 0.2500
Epoch 7/10
23/23 [==============================] - 30s 1s/step - loss: 1.2924 - accuracy: 

## Overfitting Intentionally

In [ ]:
ablation_size=None
batch_size=30
num_epochs=10

num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
print ('# epochs =', num_epochs)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

print('steps_per_epoch: {}, validation_steps: {}'.format(steps_per_epoch, validation_steps))

train_generator = generator(data_dir_train, train_doc, dim=(120,120), batch_size=batch_size, ablation=ablation_size, samples=list(range(0,30,3)))
val_generator = generator(data_dir_val, val_doc, dim=(120,120), batch_size=batch_size, ablation=ablation_size, samples=list(range(0,30,3)))

In [59]:
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=[], validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

## Final Experiment

In [ ]:
ablation_size=None
batch_size=30
num_epochs=20

num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
print ('# epochs =', num_epochs)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

print('steps_per_epoch: {}, validation_steps: {}'.format(steps_per_epoch, validation_steps))

train_generator = generator(data_dir_train, train_doc, dim=(120,120), batch_size=batch_size, ablation=ablation_size, samples=list(range(0,30,3)))
val_generator = generator(data_dir_val, val_doc, dim=(120,120), batch_size=batch_size, ablation=ablation_size, samples=list(range(0,30,3)))

In [ ]:
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=[], validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)